In [32]:
import copy
from operator import itemgetter
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import random
import time


class BestFitPriority:
    def __init__(self, is_consider_binNum):
        self.binNum = is_consider_binNum # Number of bins available
        self.h = 0  # Height required for the lowest left-aligned best-fit algorithm
        self.put_in = []  # List of items that will fit in the bin   
        self.put_position = [] # A collection of contour lines generated during packing

    def full_fit_first(self, parts, position):
        """
        Select the lowest horizontal line, position.k, among the loadable contour lines, and if there are more than one line segment, select the leftmost one first.
        From the rectangles to be loaded, compare the rectangles with position.k in order of loading, if there is a line whose width or height is equal to the width of the segment position.k.w
        If there is a rectangle whose width or height is equal to the width of the segment position.k.w, and which fills just to the left or to the right after loading, it is loaded first.
        :param parts: list of rectangles to be binned [{},{},{}].
        :param position: list of sets of contour lines [{},{},{}].
        :return.
        """
        position = sorted(position, key=itemgetter("y", "x"))
        wide = position[0]['w']
        for part in parts:
            if part['wide'] == wide:
                for pos in range(0, len(position)):
                    if position[0]['x'] == position[pos]['x'] + position[pos]['w'] and\
                                            position[0]['y'] + part['length'] == position[pos]['y']:
                        self.put_in.append({'x': position[0]['x'], 'y': position[0]['y'], 'w': part['wide'],
                                            'h': part['length'], 'angle': 0})
                        position[pos]['w'] += wide
                        position.remove(position[0])
                        position = sorted(position, key=itemgetter("y"), reverse=True)
                        self.h = position[0]['y']
                        return 1, part, position
                    elif position[0]['x'] + part['wide'] == position[pos]['x'] and\
                                            position[0]['y'] + part['length'] == position[pos]['y']:
                        self.put_in.append({'x': position[0]['x'], 'y': position[0]['y'], 'w': part['wide'],
                                            'h': part['length'], 'angle': 0})
                        position[pos]['x'] = position[0]['x']
                        position[pos]['w'] += wide
                        position.remove(position[0])
                        position = sorted(position, key=itemgetter("y"), reverse=True)
                        self.h = position[0]['y']
                        return 1, part, position
            elif part['is_fixed_direction'] == 0 and part['length'] == wide:
                for pos in range(0, len(position)):
                    if position[0]['x'] == position[pos]['x'] + position[pos]['w'] and\
                                            position[0]['y'] + part['wide'] == position[pos]['y']:
                        self.put_in.append({'x': position[0]['x'], 'y': position[0]['y'], 'w': part['wide'],
                                            'h': part['length'], 'angle': 90})
                        position[pos]['w'] += wide
                        position.remove(position[0])
                        position = sorted(position, key=itemgetter("y"), reverse=True)
                        self.h = position[0]['y']
                        return 1, part, position
                    elif position[0]['x'] + part['length'] == position[pos]['x'] and\
                                            position[0]['y'] + part['wide'] == position[pos]['y']:
                        self.put_in.append({'x': position[0]['x'], 'y': position[0]['y'], 'w': part['wide'],
                                            'h': part['length'], 'angle': 90})
                        position[pos]['x'] = position[0]['x']
                        position[pos]['w'] += wide
                        position = sorted(position, key=itemgetter("y"), reverse=True)
                        self.h = position[0]['y']
                        return 1, part, position
        return 0, [], position

    def width_fit_first(self, parts, position):
        """
        Priority is given to rectangles whose width or height is equal to the lowest horizontal line ek, if there are more than one matching rectangles, then the one with the largest area will be loaded first.
        :param parts: list of rectangles to be loaded into the bin [{w:c, h:d, is_fixed_direction:e},{},{}].
        :param position:list of contour line sets [{x:a , y:b, w:c},{},{}}
        :return.
        Indicates if it can be binned 0 means it can't 1 means it can be binned
        Rectangular part that can be binned = {x:a, y:b, w:c, h:d, angle:e}
        """
        position = sorted(position, key=itemgetter("y", "x"))
        wide = position[0]['w']
        temp = []
        for part in parts:
            if part['wide'] == wide:
                temp.append(part)
        if len(temp) == 0:
            for part in parts:
                if part['is_fixed_direction'] == 0 and part['length'] == wide:
                    temp.append(part)
            if len(temp) == 0:
                return 0, [], position
            else:
                temp = sorted(temp, key=itemgetter('wide'), reverse=True)
                position[0]['y'] += temp[0]['wide']
                position = sorted(position, key=itemgetter("y"), reverse=True)
                self.h = position[0]['y']
                self.put_in.append({'x': position[0]['x'], 'y': position[0]['y'], 'w': temp[0]['wide'],
                                    'h': temp[0]['length'], 'angle': 90})
                return 1, temp[0], position
        else:
            temp = sorted(temp, key=itemgetter('length'), reverse=True)
            position[0]['y'] += temp[0]['length']
            position = sorted(position, key=itemgetter("y"), reverse=True)
            self.h = position[0]['y']
            self.put_in.append({'x': position[0]['x'], 'y': position[0]['y'], 'w': temp[0]['wide'],
                                'h': temp[0]['length'], 'angle': 0})
            return 1, temp[0], position

    def height_fit_first(self, parts, position):
        """
        Query the rectangle whose width or height is not greater than the width of the lowest horizontal line ek and can be left-filled after loading according to the sequence of loading, if it exists, then load the first rectangle queried.
        :param parts: list of rectangles to be binned [{w:c, h:d, is_fixed_direction:e},{}, {}]
        :param position:list of contour line sets [{x:a , y:b, w:c},{},{}]
        :return.
        Indicates if it can be binned 0 means it can't 1 means it can be binned
        Rectangular part that can be binned = {x:a, y:b, w:c, h:d, angle:e}
        """
        position = sorted(position, key=itemgetter("y", "x"))
        wide = position[0]['w']
        temp = []
        for part in parts:
            if part['wide'] < wide:
                temp.append(part)
        if len(temp) == 0:
            for part in parts:
                if part['is_fixed_direction'] == 0 and part['length'] < wide:
                    temp.append(part)
            if len(temp) == 0:
                return 0, {}, position
            else:                                   # @
                for t in temp:
                    for pos in range(0, len(position)):
                        if position[pos]['x'] + position[pos]['w'] == position[0]['x'] and\
                                                position[0]['y'] + t['wide'] == position[pos]['y']:
                            self.put_in.append({'x': position[0]['x'], 'y': position[0]['y'], 'w': t['wide'],
                                                'h': t['length'], 'angle': 90})
                            position[pos]['w'] += t['length']
                            position[0]['x'] += t['length']
                            position[0]['w'] -= t['length']
                            position = sorted(position, key=itemgetter("y"), reverse=True)
                            self.h = position[0]['y']
                            return 1, t, position
                        elif position[0]['x'] + position[0]['w'] == position[pos]['x'] and\
                                                position[0]['y'] + t['wide'] == position[pos]['y']:
                            self.put_in.append({'x': position[pos]['x']-t['length'], 'y': position[0]['y'],
                                                'w': t['wide'], 'h': t['length'], 'angle': 90})
                            position[0]['w'] -= t['length']
                            position[pos]['x'] -= t['length']
                            position[pos]['w'] += t['length']
                            position = sorted(position, key=itemgetter("y"), reverse=True)
                            self.h = position[0]['y']
                            return 1, t, position
                return 0, {}, position

        else:
            for t in temp:
                for pos in range(0, len(position)):
                    if position[pos]['x'] + position[pos]['w'] == position[0]['x'] and\
                                            position[0]['y'] + t['length'] == position[pos]['y']:
                        self.put_in.append({'x': position[0]['x'], 'y': position[0]['y'], 'w': t['wide'],
                                            'h': t['length'], 'angle': 0})
                        position[pos]['w'] += t['wide']
                        position[0]['x'] += t['wide']
                        position[0]['w'] -= t['wide']
                        position = sorted(position, key=itemgetter("y"), reverse=True)
                        self.h = position[0]['y']
                        return 1, t, position
                    elif position[0]['x'] + position[0]['w'] == position[pos]['x'] and\
                                            position[0]['y'] + t['length'] == position[pos]['y']:
                        self.put_in.append({'x': position[pos]['x'] - t['wide'], 'y': position[0]['y'], 'w': t['wide'],
                                            'h': t['length'], 'angle': 0})
                        position[0]['w'] -= t['wide']
                        position[pos]['x'] -= t['wide']
                        position[pos]['w'] += t['wide']
                        position = sorted(position, key=itemgetter("y"), reverse=True)
                        self.h = position[0]['y']
                        return 1, t, position
            return 0, {}, position

    def joint_width_fit_first(self, parts, position):
        """
        Combine two rectangles in a loading sequence, if the width of the combined rectangle is equal to the width of the lowest horizontal line ek, then the first rectangle in the combining sequence will be loaded first.
        :param parts: list of rectangles to be binned [{w:c, h:d, is_fixed_direction:e},{}, {}]
        :param position:list of contour line sets [{x:a , y:b, w:c},{},{}]
        :return.
        Indicates if the bin can be entered 0 means it cannot 1 means it can be entered
        Rectangles that can be binned part = {x:a, y:b, w:c, h:d, angle:e}
        """
        position = sorted(position, key=itemgetter("y", "x"))
        wide = position[0]['w']
        for part in parts:
            for pt in parts:
                if part != pt and part['wide'] + pt['wide'] == wide:
                    self.put_in.append({'x': position[0]['x'], 'y': position[0]['y'], 'w': part['wide'],
                                        'h': part['length'], 'angle': 0})
                    height = position[0]['y']
                    position[0]['y'] += part['length']
                    position[0]['w'] = part['wide']
                    position.append({'x': position[0]['x'] + position[0]['w'], 'y': height,
                                     'w': pt['wide']})
                    position = sorted(position, key=itemgetter("y"), reverse=True)
                    self.h = position[0]['y']
                    return 1, part, position
                elif part != pt and part['is_fixed_direction'] == 0 and part['length'] + pt['wide'] == wide:
                    self.put_in.append({'x': position[0]['x'], 'y': position[0]['y'], 'w': part['wide'],
                                        'h': part['length'], 'angle': 90})
                    height = position[0]['y']
                    position[0]['y'] += part['wide']
                    position[0]['w'] = part['length']
                    position.append({'x': position[0]['x'] + position[0]['w'], 'y': height,
                                     'w': pt['wide']})
                    position = sorted(position, key=itemgetter("y"), reverse=True)
                    self.h = position[0]['y']
                    return 1, part, position
                elif part != pt and pt['is_fixed_direction'] == 0 and part['wide'] + pt['length'] == wide:
                    self.put_in.append({'x': position[0]['x'], 'y': position[0]['y'], 'w': part['wide'],
                                        'h': part['length'], 'angle': 0})
                    height = position[0]['y']
                    position[0]['y'] += part['length']
                    position[0]['w'] = part['wide']
                    position.append({'x': position[0]['x'] + position[0]['w'], 'y': height,
                                     'w': pt['length']})
                    position = sorted(position, key=itemgetter("y"), reverse=True)
                    self.h = position[0]['y']
                    return 1, part, position
                elif part != pt and pt['is_fixed_direction'] == part['is_fixed_direction'] == 0 and part['length'] + pt['length'] == wide:
                    self.put_in.append({'x': position[0]['x'], 'y': position[0]['y'], 'w': part['wide'],
                                        'h': part['length'], 'angle': 90})
                    height = position[0]['y']
                    position[0]['y'] += part['wide']
                    position[0]['w'] = part['length']
                    position.append({'x': position[0]['x'] + position[0]['w'], 'y': height,
                                     'w': pt['length']})
                    position = sorted(position, key=itemgetter("y"), reverse=True)
                    self.h = position[0]['y']
                    return 1, part, position
        return 0, {}, position

    def place_first(self, parts, position):
        """
        Within a certain range, from the rectangular parts to be loaded, according to the loading sequence, find the rectangle whose width or height is not greater than the width of the lowest horizontal line ek, and if it exists, load it; if more than one exists, load the rectangle with the largest area.
        If there is more than one, the rectangle with the largest area is loaded.
        :param parts: list of rectangles to be binned [{w:c, h:d, is_fixed_direction:e},{}, {}]
        :param position:list of contour line sets [{x:a , y:b, w:c},{},{}]
        :return.
        Indicates if the bin can be entered 0 means no 1 means yes
        Rectangles that can be binned part = {x:a, y:b, w:c, h:d, angle:e}
        """
        position = sorted(position, key=itemgetter("y", "x"))
        wide = position[0]['w']
        temp = []
        for part in parts:
            if part['wide'] < wide:
                part['area'] = part['length'] * part['wide']
                temp.append(part)
        if len(temp) == 0:
            for part in parts:
                if part['is_fixed_direction'] == 0 and part['length'] < wide:
                    part['area'] = part['length'] * part['wide']
                    temp.append(part)
            if len(temp) == 0:
                return 0, {}, position
            else:
                temp = sorted(temp, key=itemgetter('area'), reverse=True)
                self.put_in.append({'x': position[0]['x'], 'y': position[0]['y'], 'w': temp[0]['wide'],
                                    'h': temp[0]['length'], 'angle': 90})
                position.append({'x': position[0]['x'] + temp[0]['length'], 'y': position[0]['y'],
                                 'w': position[0]['w'] - temp[0]['length']})
                position[0]['y'] += temp[0]['wide']
                position[0]['w'] = temp[0]['length']
                position = sorted(position, key=itemgetter("y"), reverse=True)
                self.h = position[0]['y']
                return 1, temp[0], position
        else:
            temp = sorted(temp, key=itemgetter('area'), reverse=True)
            self.put_in.append({'x': position[0]['x'], 'y': position[0]['y'], 'w': temp[0]['wide'],
                                'h': temp[0]['length'], 'angle': 0})
            position.append({'x': position[0]['x']+temp[0]['wide'], 'y': position[0]['y'],
                             'w': position[0]['w']-temp[0]['wide']})
            position[0]['y'] += temp[0]['length']
            position[0]['w'] = temp[0]['wide']
            position = sorted(position, key=itemgetter("y"), reverse=True)
            self.h = position[0]['y']
            return 1, temp[0], position

    def llabf(self, parts, position):
        """
        :param parts: find a suitable rectangle in parts to put into position
        :param position: set of contour lines
        :return: Returns the proper rectangle and the new set of contour lines.
        """
        indic, part, e = self.full_fit_first(parts, position)
        if indic == 1:
            print(type(parts), type(e))
            print('full_fit_first match successfully', part, e)
            parts.remove(part)
            print("The rectangles that haven't been put in yet are：", parts, "\n")
            return [part], e
        else:
            indic, part, e = self.width_fit_first(parts, position)
            if indic == 1:
                print(type(parts), type(e))
                print('width_fit_first match successfully', part, e)
                parts.remove(part)
                print("The rectangles that haven't been put in yet are：", parts, "\n")
                return [part], e
            else:
                indic, part, e = self.height_fit_first(parts, position)
                if indic == 1:
                    print(type(parts), type(e))
                    print('height_fit_first match successfully', part, e)
                    parts.remove(part)
                    print("The rectangles that haven't been put in yet are：", parts, "\n")
                    return [part], e
                else:
                    indic, part, e = self.joint_width_fit_first(parts, position)
                    if indic == 1:
                        print(type(parts), type(e))
                        print('joint_width_fit_first match successfully', part, e)
                        parts.remove(part)
                        print("The rectangles that haven't been put in yet are：", parts, "\n")
                        return [part], e
                    else:
                        indic, part, e = self.place_first(parts, position)
                        if indic == 1:
                            print(type(parts), type(e))
                            print('place_first match successfully', part, e)
                            parts.remove(part)
                            print("The rectangles that haven't been put in yet are：", parts, "\n")
                            return [part], e
                        else:
                            return [], e

    def one_binNum(self, part_list, total_area):
        """
        Determine if all the rectangles in part_list fit into the total_area space.
        :param part_list:[{'length':a , 'wide':b, 'height':h, 'is_fixed_direction':0/1}] 0 means there is no direction requirement 1 means there is a direction requirement
        :param total_area:[{'length':l, 'wide':w, 'height':h, 'can_multi_binNum':f}]]
        :return: Rectangles put in and rectangles not put in
        """
        parts = copy.deepcopy(part_list)
        position = [{'x': 0, 'y': 0, 'w': total_area['wide']}]
        self.h = 0
        solved = []
        unsolved = []
        while len(parts) and self.h <= total_area['length']:
            part, position = self.llabf(parts, position)
            if self.h <= total_area['length'] and len(part) != 0:
                solved.append(part[0])
                unsolved = parts
            else:
                return unsolved, solved
        else:
            return unsolved, solved

    def controller(self, part_list, total_area):
        """
        There are N (N>=1) bins, determine if the rectangles in part_list can all fit into the total_area of the N bins.
        :param part_list: [{'length':a , 'wide':b, 'height':h, 'is_fixed_direction':0/1}] 0 means no direction required 1 means direction required
        :param total_area: [{'length':l, 'wide':w, 'height':h, 'can_multi_binNum':f}]
        :return: Indication of whether or not it can be entered 0 means it can't 1 means it can be entered
        """
        parts = copy.deepcopy(part_list)
        count = self.binNum
        while count and len(parts):
            unsolved, solved = self.one_binNum(parts, total_area)
            parts = unsolved
            count -= 1
        else:
            if len(parts) == 0:
                return 1
            else:
                return 0


# C1-P1

In [57]:
def C1P1():
    total_area = {'length': 20, 'wide': 20}
    P1_C1 = [
    [2, 12], [7, 12], [8, 6], [3, 6], [3, 5], [5, 5], [3, 12], [3, 7],
    [5, 7], [2, 6], [3, 2], [4, 2], [3, 4], [4, 4], [9, 2], [11, 2]
]

    part_list = [{'length': item[1], 'wide': item[0], 'is_fixed_direction': 1} for item in P1_C1]
    a = BestFitPriority(1)
    
    start_time = time.time()
    
    # Call the controller function to get the packing results
    t = a.controller(part_list, total_area)
    print(t)
    
    print("----------------------------------------------------------\n")
    unsolved, solved = a.one_binNum(part_list, total_area)
    print("Unsolved parts:\n", unsolved)
    print("Solved parts:\n", solved)
    
    # Calculate the area of solved parts
    solved_area = sum(part['length'] * part['wide'] for part in solved)
    
    # Calculate the utilization
    utilization = solved_area / (total_area['length'] * total_area['wide'])
    print("Utilization:", utilization)
    
    end_time = time.time()  # Record the end time
    execution_time = end_time - start_time  # Calculate the execution time
    print("Execution Time:", execution_time, "seconds")

if __name__ == '__main__':
    C1P1()

<class 'list'> <class 'list'>
joint_width_fit_first match successfully {'length': 2, 'wide': 9, 'is_fixed_direction': 1} [{'x': 0, 'y': 2, 'w': 9}, {'x': 9, 'y': 0, 'w': 11}]
The rectangles that haven't been put in yet are： [{'length': 12, 'wide': 2, 'is_fixed_direction': 1}, {'length': 12, 'wide': 7, 'is_fixed_direction': 1}, {'length': 6, 'wide': 8, 'is_fixed_direction': 1}, {'length': 6, 'wide': 3, 'is_fixed_direction': 1}, {'length': 5, 'wide': 3, 'is_fixed_direction': 1}, {'length': 5, 'wide': 5, 'is_fixed_direction': 1}, {'length': 12, 'wide': 3, 'is_fixed_direction': 1}, {'length': 7, 'wide': 3, 'is_fixed_direction': 1}, {'length': 7, 'wide': 5, 'is_fixed_direction': 1}, {'length': 6, 'wide': 2, 'is_fixed_direction': 1}, {'length': 2, 'wide': 3, 'is_fixed_direction': 1}, {'length': 2, 'wide': 4, 'is_fixed_direction': 1}, {'length': 4, 'wide': 3, 'is_fixed_direction': 1}, {'length': 4, 'wide': 4, 'is_fixed_direction': 1}, {'length': 2, 'wide': 11, 'is_fixed_direction': 1}] 

<cla

# C1-P2

In [56]:
def C1P2():
    total_area = {'length': 20, 'wide': 20}
    P1_C2 = [
    [4, 1], [4, 5], [9, 4], [3, 5], [3, 9], [1, 4], [5, 3], [4, 1],
    [5, 5], [7, 2], [9, 3], [3, 13], [2, 8], [15, 4], [5, 4], [10, 6],
    [7, 2]]

    part_list = [{'length': item[1], 'wide': item[0], 'is_fixed_direction': 1} for item in P1_C2]
    a = BestFitPriority(1)
    
    start_time = time.time()
    
    # Call the controller function to get the packing results
    t = a.controller(part_list, total_area)
    print(t)
    
    print("----------------------------------------------------------\n")
    unsolved, solved = a.one_binNum(part_list, total_area)
    print("Unsolved parts:\n", unsolved)
    print("Solved parts:\n", solved)

    # Calculate the area of solved parts
    solved_area = sum(part['length'] * part['wide'] for part in solved)
    
    # Calculate the utilization
    utilization = solved_area / (total_area['length'] * total_area['wide'])
    print("Utilization:", utilization)
    
    end_time = time.time()  # Record the end time
    execution_time = end_time - start_time  # Calculate the execution time
    print("Execution Time:", execution_time, "seconds")


if __name__ == '__main__':
    C1P2()

<class 'list'> <class 'list'>
joint_width_fit_first match successfully {'length': 3, 'wide': 5, 'is_fixed_direction': 1} [{'x': 0, 'y': 3, 'w': 5}, {'x': 5, 'y': 0, 'w': 15}]
The rectangles that haven't been put in yet are： [{'length': 1, 'wide': 4, 'is_fixed_direction': 1}, {'length': 5, 'wide': 4, 'is_fixed_direction': 1}, {'length': 4, 'wide': 9, 'is_fixed_direction': 1}, {'length': 5, 'wide': 3, 'is_fixed_direction': 1}, {'length': 9, 'wide': 3, 'is_fixed_direction': 1}, {'length': 4, 'wide': 1, 'is_fixed_direction': 1}, {'length': 1, 'wide': 4, 'is_fixed_direction': 1}, {'length': 5, 'wide': 5, 'is_fixed_direction': 1}, {'length': 2, 'wide': 7, 'is_fixed_direction': 1}, {'length': 3, 'wide': 9, 'is_fixed_direction': 1}, {'length': 13, 'wide': 3, 'is_fixed_direction': 1}, {'length': 8, 'wide': 2, 'is_fixed_direction': 1}, {'length': 4, 'wide': 15, 'is_fixed_direction': 1}, {'length': 4, 'wide': 5, 'is_fixed_direction': 1}, {'length': 6, 'wide': 10, 'is_fixed_direction': 1}, {'lengt

# C1-P3

In [55]:
def C1P3():
    total_area = {'length': 20, 'wide': 20}
    C1_P3 = [
    [4, 14], [5, 2], [2, 2], [9, 7], [5, 5], [2, 5], [7, 7], [3, 5],
    [6, 5], [3, 2], [6, 2], [4, 6], [6, 3], [10, 3], [6, 3], [10, 3]
]

    part_list = [{'length': item[1], 'wide': item[0], 'is_fixed_direction': 1} for item in C1_P3]
    a = BestFitPriority(1)
    
    start_time = time.time()
    
    # Call the controller function to get the packing results
    t = a.controller(part_list, total_area)
    print(t)
    
    print("----------------------------------------------------------\n")
    unsolved, solved = a.one_binNum(part_list, total_area)
    print("Unsolved parts:\n", unsolved)
    print("Solved parts:\n", solved)

    # Calculate the area of solved parts
    solved_area = sum(part['length'] * part['wide'] for part in solved)
    
    # Calculate the utilization
    utilization = solved_area / (total_area['length'] * total_area['wide'])
    print("Utilization:", utilization)
    
    end_time = time.time()  # Record the end time
    execution_time = end_time - start_time  # Calculate the execution time
    print("Execution Time:", execution_time, "seconds")


if __name__ == '__main__':
    C1P3()

<class 'list'> <class 'list'>
place_first match successfully {'length': 7, 'wide': 9, 'is_fixed_direction': 1, 'area': 63} [{'x': 0, 'y': 7, 'w': 9}, {'x': 9, 'y': 0, 'w': 11}]
The rectangles that haven't been put in yet are： [{'length': 14, 'wide': 4, 'is_fixed_direction': 1, 'area': 56}, {'length': 2, 'wide': 5, 'is_fixed_direction': 1, 'area': 10}, {'length': 2, 'wide': 2, 'is_fixed_direction': 1, 'area': 4}, {'length': 5, 'wide': 5, 'is_fixed_direction': 1, 'area': 25}, {'length': 5, 'wide': 2, 'is_fixed_direction': 1, 'area': 10}, {'length': 7, 'wide': 7, 'is_fixed_direction': 1, 'area': 49}, {'length': 5, 'wide': 3, 'is_fixed_direction': 1, 'area': 15}, {'length': 5, 'wide': 6, 'is_fixed_direction': 1, 'area': 30}, {'length': 2, 'wide': 3, 'is_fixed_direction': 1, 'area': 6}, {'length': 2, 'wide': 6, 'is_fixed_direction': 1, 'area': 12}, {'length': 6, 'wide': 4, 'is_fixed_direction': 1, 'area': 24}, {'length': 3, 'wide': 6, 'is_fixed_direction': 1, 'area': 18}, {'length': 3, 'wid

# C2-P1

In [54]:
def C2P1():
    total_area = {'length': 15, 'wide': 40}
    C2_P1 = [
    [11, 3], [13, 3], [9, 2], [7, 2], [9, 3], [7, 3], [11, 2], [13, 2],
    [11, 4], [13, 4], [3, 5], [11, 2], [2, 2], [11, 3], [2, 3], [5, 4],
    [6, 4], [12, 2], [1, 2], [3, 5], [13, 5], [12, 4], [1, 4], [5, 2], [6, 2]
]

    part_list = [{'length': item[1], 'wide': item[0], 'is_fixed_direction': 1} for item in C2_P1]
    a = BestFitPriority(1)
    
    start_time = time.time()
    
    # Call the controller function to get the packing results
    t = a.controller(part_list, total_area)
    print(t)
    
    print("----------------------------------------------------------\n")
    unsolved, solved = a.one_binNum(part_list, total_area)
    print("Unsolved parts:\n", unsolved)
    print("Solved parts:\n", solved)

    # Calculate the area of solved parts
    solved_area = sum(part['length'] * part['wide'] for part in solved)
    
    # Calculate the utilization
    utilization = solved_area / (total_area['length'] * total_area['wide'])
    print("Utilization:", utilization)
    
    end_time = time.time()  # Record the end time
    execution_time = end_time - start_time  # Calculate the execution time
    print("Execution Time:", execution_time, "seconds")


if __name__ == '__main__':
    C2P1()

<class 'list'> <class 'list'>
place_first match successfully {'length': 5, 'wide': 13, 'is_fixed_direction': 1, 'area': 65} [{'x': 0, 'y': 5, 'w': 13}, {'x': 13, 'y': 0, 'w': 27}]
The rectangles that haven't been put in yet are： [{'length': 3, 'wide': 11, 'is_fixed_direction': 1, 'area': 33}, {'length': 3, 'wide': 13, 'is_fixed_direction': 1, 'area': 39}, {'length': 2, 'wide': 9, 'is_fixed_direction': 1, 'area': 18}, {'length': 2, 'wide': 7, 'is_fixed_direction': 1, 'area': 14}, {'length': 3, 'wide': 9, 'is_fixed_direction': 1, 'area': 27}, {'length': 3, 'wide': 7, 'is_fixed_direction': 1, 'area': 21}, {'length': 2, 'wide': 11, 'is_fixed_direction': 1, 'area': 22}, {'length': 2, 'wide': 13, 'is_fixed_direction': 1, 'area': 26}, {'length': 4, 'wide': 11, 'is_fixed_direction': 1, 'area': 44}, {'length': 4, 'wide': 13, 'is_fixed_direction': 1, 'area': 52}, {'length': 5, 'wide': 3, 'is_fixed_direction': 1, 'area': 15}, {'length': 2, 'wide': 11, 'is_fixed_direction': 1, 'area': 22}, {'lengt

# C2-P2

In [53]:
def C2P2():
    total_area = {'length': 15, 'wide': 40}
    C2_P2 = [
    [11, 2], [2, 3], [10, 7], [8, 4], [9, 5], [7, 2], [4, 1], [6, 1],
    [4, 5], [8, 3], [1, 3], [5, 5], [3, 1], [12, 4], [6, 2], [2, 4],
    [11, 4], [10, 2], [3, 2], [11, 2], [3, 4], [26, 4], [8, 4], [3, 2],
    [6, 2]
]

    part_list = [{'length': item[1], 'wide': item[0], 'is_fixed_direction': 1} for item in C2_P2]
    a = BestFitPriority(1)
    
    start_time = time.time()

    
    # Call the controller function to get the packing results
    t = a.controller(part_list, total_area)
    print(t)
    
    print("----------------------------------------------------------\n")
    unsolved, solved = a.one_binNum(part_list, total_area)
    print("Unsolved parts:\n", unsolved)
    print("Solved parts:\n", solved)

    # Calculate the area of solved parts
    solved_area = sum(part['length'] * part['wide'] for part in solved)
    
    # Calculate the utilization
    utilization = solved_area / (total_area['length'] * total_area['wide'])
    print("Utilization:", utilization)
    
    end_time = time.time()  # Record the end time
    execution_time = end_time - start_time  # Calculate the execution time
    print("Execution Time:", execution_time, "seconds")


if __name__ == '__main__':
    C2P2()

<class 'list'> <class 'list'>
place_first match successfully {'length': 4, 'wide': 26, 'is_fixed_direction': 1, 'area': 104} [{'x': 0, 'y': 4, 'w': 26}, {'x': 26, 'y': 0, 'w': 14}]
The rectangles that haven't been put in yet are： [{'length': 2, 'wide': 11, 'is_fixed_direction': 1, 'area': 22}, {'length': 3, 'wide': 2, 'is_fixed_direction': 1, 'area': 6}, {'length': 7, 'wide': 10, 'is_fixed_direction': 1, 'area': 70}, {'length': 4, 'wide': 8, 'is_fixed_direction': 1, 'area': 32}, {'length': 5, 'wide': 9, 'is_fixed_direction': 1, 'area': 45}, {'length': 2, 'wide': 7, 'is_fixed_direction': 1, 'area': 14}, {'length': 1, 'wide': 4, 'is_fixed_direction': 1, 'area': 4}, {'length': 1, 'wide': 6, 'is_fixed_direction': 1, 'area': 6}, {'length': 5, 'wide': 4, 'is_fixed_direction': 1, 'area': 20}, {'length': 3, 'wide': 8, 'is_fixed_direction': 1, 'area': 24}, {'length': 3, 'wide': 1, 'is_fixed_direction': 1, 'area': 3}, {'length': 5, 'wide': 5, 'is_fixed_direction': 1, 'area': 25}, {'length': 1, '

# C2-P3

In [52]:
def C2P3():
    total_area = {'length': 15, 'wide': 40}
    C2_P3 = [
    [12, 7], [7, 7], [7, 1], [5, 1], [3, 2], [6, 2], [7, 2], [5, 2],
    [3, 1], [6, 1], [12, 6], [9, 6], [12, 2], [7, 2], [10, 3], [4, 1],
    [5, 1], [16, 3], [5, 3], [11, 2], [3, 2], [10, 3], [9, 3], [16, 3],
    [5, 3]
]

    part_list = [{'length': item[1], 'wide': item[0], 'is_fixed_direction': 1} for item in C2_P3]
    a = BestFitPriority(1)
    
    start_time = time.time()
    
    # Call the controller function to get the packing results
    t = a.controller(part_list, total_area)
    print(t)
    
    print("----------------------------------------------------------\n")
    unsolved, solved = a.one_binNum(part_list, total_area)
    print("Unsolved parts:\n", unsolved)
    print("Solved parts:\n", solved)

    # Calculate the area of solved parts
    solved_area = sum(part['length'] * part['wide'] for part in solved)
    
    # Calculate the utilization
    utilization = solved_area / (total_area['length'] * total_area['wide'])
    print("Utilization:", utilization)
    
    end_time = time.time()  # Record the end time
    execution_time = end_time - start_time  # Calculate the execution time
    print("Execution Time:", execution_time, "seconds")


if __name__ == '__main__':
    C2P3()

<class 'list'> <class 'list'>
place_first match successfully {'length': 7, 'wide': 12, 'is_fixed_direction': 1, 'area': 84} [{'x': 0, 'y': 7, 'w': 12}, {'x': 12, 'y': 0, 'w': 28}]
The rectangles that haven't been put in yet are： [{'length': 7, 'wide': 7, 'is_fixed_direction': 1, 'area': 49}, {'length': 1, 'wide': 7, 'is_fixed_direction': 1, 'area': 7}, {'length': 1, 'wide': 5, 'is_fixed_direction': 1, 'area': 5}, {'length': 2, 'wide': 3, 'is_fixed_direction': 1, 'area': 6}, {'length': 2, 'wide': 6, 'is_fixed_direction': 1, 'area': 12}, {'length': 2, 'wide': 7, 'is_fixed_direction': 1, 'area': 14}, {'length': 2, 'wide': 5, 'is_fixed_direction': 1, 'area': 10}, {'length': 1, 'wide': 3, 'is_fixed_direction': 1, 'area': 3}, {'length': 1, 'wide': 6, 'is_fixed_direction': 1, 'area': 6}, {'length': 6, 'wide': 12, 'is_fixed_direction': 1, 'area': 72}, {'length': 6, 'wide': 9, 'is_fixed_direction': 1, 'area': 54}, {'length': 2, 'wide': 12, 'is_fixed_direction': 1, 'area': 24}, {'length': 2, 'wi

# C3-P1

In [51]:
def C3P1():
    total_area = {'length': 30, 'wide': 60}
    C3_P1 = [
    [7, 5], [14, 5], [14, 8], [4, 8], [21, 13], [7, 11], [14, 11], [14, 5],
    [4, 5], [18, 3], [21, 3], [17, 11], [4, 11], [7, 4], [5, 4], [6, 7],
    [18, 5], [3, 5], [7, 3], [5, 3], [18, 4], [3, 4], [12, 2], [6, 2],
    [18, 5], [21, 5], [17, 3], [4, 3]
]

    part_list = [{'length': item[1], 'wide': item[0], 'is_fixed_direction': 1} for item in C3_P1]
    a = BestFitPriority(1)
    
    start_time = time.time()
    
    # Call the controller function to get the packing results
    t = a.controller(part_list, total_area)
    print(t)
    
    print("----------------------------------------------------------\n")
    unsolved, solved = a.one_binNum(part_list, total_area)
    print("Unsolved parts:\n", unsolved)
    print("Solved parts:\n", solved)

    # Calculate the area of solved parts
    solved_area = sum(part['length'] * part['wide'] for part in solved)
    
    # Calculate the utilization
    utilization = solved_area / (total_area['length'] * total_area['wide'])
    print("Utilization:", utilization)
    
    end_time = time.time()  # Record the end time
    execution_time = end_time - start_time  # Calculate the execution time
    print("Execution Time:", execution_time, "seconds")


if __name__ == '__main__':
    C3P1()

<class 'list'> <class 'list'>
place_first match successfully {'length': 13, 'wide': 21, 'is_fixed_direction': 1, 'area': 273} [{'x': 0, 'y': 13, 'w': 21}, {'x': 21, 'y': 0, 'w': 39}]
The rectangles that haven't been put in yet are： [{'length': 5, 'wide': 7, 'is_fixed_direction': 1, 'area': 35}, {'length': 5, 'wide': 14, 'is_fixed_direction': 1, 'area': 70}, {'length': 8, 'wide': 14, 'is_fixed_direction': 1, 'area': 112}, {'length': 8, 'wide': 4, 'is_fixed_direction': 1, 'area': 32}, {'length': 11, 'wide': 7, 'is_fixed_direction': 1, 'area': 77}, {'length': 11, 'wide': 14, 'is_fixed_direction': 1, 'area': 154}, {'length': 5, 'wide': 14, 'is_fixed_direction': 1, 'area': 70}, {'length': 5, 'wide': 4, 'is_fixed_direction': 1, 'area': 20}, {'length': 3, 'wide': 18, 'is_fixed_direction': 1, 'area': 54}, {'length': 3, 'wide': 21, 'is_fixed_direction': 1, 'area': 63}, {'length': 11, 'wide': 17, 'is_fixed_direction': 1, 'area': 187}, {'length': 11, 'wide': 4, 'is_fixed_direction': 1, 'area': 44

# C3-P2

In [50]:
def C3P2():
    total_area = {'length': 30, 'wide': 60}
    C3_P2 = [
    [18, 6], [12, 2], [7, 10], [23, 4], [1, 4], [7, 7], [4, 11], [5, 6],
    [7, 2], [11, 6], [19, 10], [5, 11], [2, 4], [5, 7], [2, 4], [12, 7],
    [13, 7], [6, 3], [10, 6], [16, 9], [4, 1], [10, 4], [24, 6], [9, 9],
    [1, 2], [5, 8], [5, 3], [25, 7], [21, 5]
]

    part_list = [{'length': item[1], 'wide': item[0], 'is_fixed_direction': 1} for item in C3_P2]
    a = BestFitPriority(1)
    
    start_time = time.time()
    
    # Call the controller function to get the packing results
    t = a.controller(part_list, total_area)
    print(t)
    
    print("----------------------------------------------------------\n")
    unsolved, solved = a.one_binNum(part_list, total_area)
    print("Unsolved parts:\n", unsolved)
    print("Solved parts:\n", solved)

    # Calculate the area of solved parts
    solved_area = sum(part['length'] * part['wide'] for part in solved)
    
    # Calculate the utilization
    utilization = solved_area / (total_area['length'] * total_area['wide'])
    print("Utilization:", utilization)
    
    end_time = time.time()  # Record the end time
    execution_time = end_time - start_time  # Calculate the execution time
    print("Execution Time:", execution_time, "seconds")


if __name__ == '__main__':
    C3P2()

<class 'list'> <class 'list'>
place_first match successfully {'length': 10, 'wide': 19, 'is_fixed_direction': 1, 'area': 190} [{'x': 0, 'y': 10, 'w': 19}, {'x': 19, 'y': 0, 'w': 41}]
The rectangles that haven't been put in yet are： [{'length': 6, 'wide': 18, 'is_fixed_direction': 1, 'area': 108}, {'length': 2, 'wide': 12, 'is_fixed_direction': 1, 'area': 24}, {'length': 10, 'wide': 7, 'is_fixed_direction': 1, 'area': 70}, {'length': 4, 'wide': 23, 'is_fixed_direction': 1, 'area': 92}, {'length': 4, 'wide': 1, 'is_fixed_direction': 1, 'area': 4}, {'length': 7, 'wide': 7, 'is_fixed_direction': 1, 'area': 49}, {'length': 11, 'wide': 4, 'is_fixed_direction': 1, 'area': 44}, {'length': 6, 'wide': 5, 'is_fixed_direction': 1, 'area': 30}, {'length': 2, 'wide': 7, 'is_fixed_direction': 1, 'area': 14}, {'length': 6, 'wide': 11, 'is_fixed_direction': 1, 'area': 66}, {'length': 11, 'wide': 5, 'is_fixed_direction': 1, 'area': 55}, {'length': 4, 'wide': 2, 'is_fixed_direction': 1, 'area': 8}, {'len

# C3-P3

In [49]:
def C3P3():
    total_area = {'length': 30, 'wide': 60}
    C3_P3 = [
    [24, 9], [8, 9], [11, 9], [17, 9], [24, 4], [8, 4], [6, 1], [5, 1],
    [17, 4], [7, 3], [6, 3], [5, 12], [13, 12], [14, 14], [14, 2], [2, 2],
    [3, 8], [9, 8], [14, 12], [2, 12], [3, 6], [9, 6], [5, 2], [13, 2],
    [18, 3], [14, 3], [16, 3], [12, 3]
]

    part_list = [{'length': item[1], 'wide': item[0], 'is_fixed_direction': 1} for item in C3_P3]
    a = BestFitPriority(1)
    
    start_time = time.time()
    
    # Call the controller function to get the packing results
    t = a.controller(part_list, total_area)
    print(t)
    
    print("----------------------------------------------------------\n")
    unsolved, solved = a.one_binNum(part_list, total_area)
    print("Unsolved parts:\n", unsolved)
    print("Solved parts:\n", solved)

    # Calculate the area of solved parts
    solved_area = sum(part['length'] * part['wide'] for part in solved)
    
    # Calculate the utilization
    utilization = solved_area / (total_area['length'] * total_area['wide'])
    print("Utilization:", utilization)
    
    end_time = time.time()  # Record the end time
    execution_time = end_time - start_time  # Calculate the execution time
    print("Execution Time:", execution_time, "seconds")


if __name__ == '__main__':
    C3P3()

<class 'list'> <class 'list'>
place_first match successfully {'length': 9, 'wide': 24, 'is_fixed_direction': 1, 'area': 216} [{'x': 0, 'y': 9, 'w': 24}, {'x': 24, 'y': 0, 'w': 36}]
The rectangles that haven't been put in yet are： [{'length': 9, 'wide': 8, 'is_fixed_direction': 1, 'area': 72}, {'length': 9, 'wide': 11, 'is_fixed_direction': 1, 'area': 99}, {'length': 9, 'wide': 17, 'is_fixed_direction': 1, 'area': 153}, {'length': 4, 'wide': 24, 'is_fixed_direction': 1, 'area': 96}, {'length': 4, 'wide': 8, 'is_fixed_direction': 1, 'area': 32}, {'length': 1, 'wide': 6, 'is_fixed_direction': 1, 'area': 6}, {'length': 1, 'wide': 5, 'is_fixed_direction': 1, 'area': 5}, {'length': 4, 'wide': 17, 'is_fixed_direction': 1, 'area': 68}, {'length': 3, 'wide': 7, 'is_fixed_direction': 1, 'area': 21}, {'length': 3, 'wide': 6, 'is_fixed_direction': 1, 'area': 18}, {'length': 12, 'wide': 5, 'is_fixed_direction': 1, 'area': 60}, {'length': 12, 'wide': 13, 'is_fixed_direction': 1, 'area': 156}, {'leng

# C4-P1

In [48]:
def C4P1():
    total_area = {'length': 60, 'wide': 60}
    C4_P1 = [
    [2, 7], [24, 7], [16, 4], [18, 4], [16, 7], [18, 7], [2, 4], [24, 4],
    [4, 28], [6, 18], [14, 12], [2, 12], [18, 19], [9, 8], [7, 8], [9, 11],
    [7, 11], [14, 6], [2, 6], [6, 10], [16, 10], [3, 5], [4, 5], [8, 12],
    [3, 18], [3, 3], [8, 3], [5, 20], [3, 17], [3, 7], [5, 7], [3, 7],
    [4, 7], [4, 21], [10, 19], [4, 17], [8, 17], [3, 10], [5, 10], [7, 6],
    [8, 6], [15, 12], [3, 12], [11, 10], [5, 10], [4, 2], [8, 2], [10, 2], [12, 2]
]

    part_list = [{'length': item[1], 'wide': item[0], 'is_fixed_direction': 1} for item in C4_P1]
    a = BestFitPriority(1)
    
    start_time = time.time()
    
    # Call the controller function to get the packing results
    t = a.controller(part_list, total_area)
    print(t)
    
    print("----------------------------------------------------------\n")
    unsolved, solved = a.one_binNum(part_list, total_area)
    print("Unsolved parts:\n", unsolved)
    print("Solved parts:\n", solved)

    # Calculate the area of solved parts
    solved_area = sum(part['length'] * part['wide'] for part in solved)
    
    # Calculate the utilization
    utilization = solved_area / (total_area['length'] * total_area['wide'])
    print("Utilization:", utilization)
    
    end_time = time.time()  # Record the end time
    execution_time = end_time - start_time  # Calculate the execution time
    print("Execution Time:", execution_time, "seconds")


if __name__ == '__main__':
    C4P1()

<class 'list'> <class 'list'>
place_first match successfully {'length': 19, 'wide': 18, 'is_fixed_direction': 1, 'area': 342} [{'x': 0, 'y': 19, 'w': 18}, {'x': 18, 'y': 0, 'w': 42}]
The rectangles that haven't been put in yet are： [{'length': 7, 'wide': 2, 'is_fixed_direction': 1, 'area': 14}, {'length': 7, 'wide': 24, 'is_fixed_direction': 1, 'area': 168}, {'length': 4, 'wide': 16, 'is_fixed_direction': 1, 'area': 64}, {'length': 4, 'wide': 18, 'is_fixed_direction': 1, 'area': 72}, {'length': 7, 'wide': 16, 'is_fixed_direction': 1, 'area': 112}, {'length': 7, 'wide': 18, 'is_fixed_direction': 1, 'area': 126}, {'length': 4, 'wide': 2, 'is_fixed_direction': 1, 'area': 8}, {'length': 4, 'wide': 24, 'is_fixed_direction': 1, 'area': 96}, {'length': 28, 'wide': 4, 'is_fixed_direction': 1, 'area': 112}, {'length': 18, 'wide': 6, 'is_fixed_direction': 1, 'area': 108}, {'length': 12, 'wide': 14, 'is_fixed_direction': 1, 'area': 168}, {'length': 12, 'wide': 2, 'is_fixed_direction': 1, 'area': 

# C4-P2

In [47]:
def C4P2():
    total_area = {'length': 60, 'wide': 60}
    C4_P2 = [
    [10, 14], [3, 13], [28, 5], [5, 8], [14, 9], [12, 14], [13, 10], [3, 17],
    [1, 5], [4, 1], [18, 4], [1, 1], [2, 6], [4, 14], [3, 18], [4, 14],
    [8, 17], [11, 5], [9, 12], [4, 7], [25, 8], [7, 5], [24, 9], [9, 14],
    [12, 19], [2, 4], [2, 7], [3, 4], [5, 30], [5, 3], [10, 26], [6, 5],
    [4, 9], [1, 4], [9, 2], [4, 17], [5, 2], [4, 4], [6, 2], [4, 10], [2, 4],
    [3, 12], [6, 5], [3, 9], [7, 18], [6, 6], [18, 7], [13, 9], [25, 7]
]

    part_list = [{'length': item[1], 'wide': item[0], 'is_fixed_direction': 1} for item in C4_P2]
    a = BestFitPriority(1)
    
    start_time = time.time()
    
    # Call the controller function to get the packing results
    t = a.controller(part_list, total_area)
    print(t)
    
    print("----------------------------------------------------------\n")
    unsolved, solved = a.one_binNum(part_list, total_area)
    print("Unsolved parts:\n", unsolved)
    print("Solved parts:\n", solved)

    # Calculate the area of solved parts
    solved_area = sum(part['length'] * part['wide'] for part in solved)
    
    # Calculate the utilization
    utilization = solved_area / (total_area['length'] * total_area['wide'])
    print("Utilization:", utilization)
    
    end_time = time.time()  # Record the end time
    execution_time = end_time - start_time  # Calculate the execution time
    print("Execution Time:", execution_time, "seconds")


if __name__ == '__main__':
    C4P2()

<class 'list'> <class 'list'>
place_first match successfully {'length': 26, 'wide': 10, 'is_fixed_direction': 1, 'area': 260} [{'x': 0, 'y': 26, 'w': 10}, {'x': 10, 'y': 0, 'w': 50}]
The rectangles that haven't been put in yet are： [{'length': 14, 'wide': 10, 'is_fixed_direction': 1, 'area': 140}, {'length': 13, 'wide': 3, 'is_fixed_direction': 1, 'area': 39}, {'length': 5, 'wide': 28, 'is_fixed_direction': 1, 'area': 140}, {'length': 8, 'wide': 5, 'is_fixed_direction': 1, 'area': 40}, {'length': 9, 'wide': 14, 'is_fixed_direction': 1, 'area': 126}, {'length': 14, 'wide': 12, 'is_fixed_direction': 1, 'area': 168}, {'length': 10, 'wide': 13, 'is_fixed_direction': 1, 'area': 130}, {'length': 17, 'wide': 3, 'is_fixed_direction': 1, 'area': 51}, {'length': 5, 'wide': 1, 'is_fixed_direction': 1, 'area': 5}, {'length': 1, 'wide': 4, 'is_fixed_direction': 1, 'area': 4}, {'length': 4, 'wide': 18, 'is_fixed_direction': 1, 'area': 72}, {'length': 1, 'wide': 1, 'is_fixed_direction': 1, 'area': 1}

# C4-P3

In [42]:
def C4P3():
    total_area = {'length': 60, 'wide': 60}
    C4_P3 = [
    [10, 4], [12, 4], [13, 5], [3, 5], [7, 22], [6, 22], [9, 23], [10, 19],
    [3, 15], [5, 13], [2, 10], [2, 10], [13, 18], [3, 18], [2, 3], [2, 3],
    [5, 2], [4, 2], [3, 4], [9, 4], [7, 1], [6, 1], [2, 4], [20, 4], [4, 7],
    [12, 7], [9, 4], [4, 4], [9, 9], [2, 5], [20, 5], [9, 5], [4, 5],
    [4, 2], [12, 2], [3, 15], [21, 11], [11, 3], [3, 3], [11, 23],
    [11, 23], [11, 8], [3, 8], [21, 4], [14, 4], [3, 13], [35, 13],
    [11, 5], [11, 5]
]

    part_list = [{'length': item[1], 'wide': item[0], 'is_fixed_direction': 1} for item in C4_P3]
    a = BestFitPriority(1)
    
    start_time = time.time()
    
    # Call the controller function to get the packing results
    t = a.controller(part_list, total_area)
    print(t)
    
    print("----------------------------------------------------------\n")
    unsolved, solved = a.one_binNum(part_list, total_area)
    print("Unsolved parts:\n", unsolved)
    print("Solved parts:\n", solved)

    # Calculate the area of solved parts
    solved_area = sum(part['length'] * part['wide'] for part in solved)
    
    # Calculate the utilization
    utilization = solved_area / (total_area['length'] * total_area['wide'])
    print("Utilization:", utilization)
    
    end_time = time.time()  # Record the end time
    execution_time = end_time - start_time  # Calculate the execution time
    print("Execution Time:", execution_time, "seconds")


if __name__ == '__main__':
    C4P3()

<class 'list'> <class 'list'>
place_first match successfully {'length': 13, 'wide': 35, 'is_fixed_direction': 1, 'area': 455} [{'x': 0, 'y': 13, 'w': 35}, {'x': 35, 'y': 0, 'w': 25}]
The rectangles that haven't been put in yet are： [{'length': 4, 'wide': 10, 'is_fixed_direction': 1, 'area': 40}, {'length': 4, 'wide': 12, 'is_fixed_direction': 1, 'area': 48}, {'length': 5, 'wide': 13, 'is_fixed_direction': 1, 'area': 65}, {'length': 5, 'wide': 3, 'is_fixed_direction': 1, 'area': 15}, {'length': 22, 'wide': 7, 'is_fixed_direction': 1, 'area': 154}, {'length': 22, 'wide': 6, 'is_fixed_direction': 1, 'area': 132}, {'length': 23, 'wide': 9, 'is_fixed_direction': 1, 'area': 207}, {'length': 19, 'wide': 10, 'is_fixed_direction': 1, 'area': 190}, {'length': 15, 'wide': 3, 'is_fixed_direction': 1, 'area': 45}, {'length': 13, 'wide': 5, 'is_fixed_direction': 1, 'area': 65}, {'length': 10, 'wide': 2, 'is_fixed_direction': 1, 'area': 20}, {'length': 10, 'wide': 2, 'is_fixed_direction': 1, 'area': 

# N1

In [43]:
def N1():
    total_area = {'length': 40, 'wide': 40}
    N_1 = [
    [7, 6], [40, 16], [5, 20], [24, 24], [7, 4], [4, 4], [7, 8], [4, 20],
    [5, 4], [7, 6]
]

    part_list = [{'length': item[1], 'wide': item[0], 'is_fixed_direction': 1} for item in N_1]
    a = BestFitPriority(1)
    
    start_time = time.time()
    
    # Call the controller function to get the packing results
    t = a.controller(part_list, total_area)
    print(t)
    
    print("----------------------------------------------------------\n")
    unsolved, solved = a.one_binNum(part_list, total_area)
    print("Unsolved parts:\n", unsolved)
    print("Solved parts:\n", solved)

    # Calculate the area of solved parts
    solved_area = sum(part['length'] * part['wide'] for part in solved)
    
    # Calculate the utilization
    utilization = solved_area / (total_area['length'] * total_area['wide'])
    print("Utilization:", utilization)
    
    end_time = time.time()  # Record the end time
    execution_time = end_time - start_time  # Calculate the execution time
    print("Execution Time:", execution_time, "seconds")


if __name__ == '__main__':
    N1()

<class 'list'> <class 'list'>
width_fit_first match successfully {'length': 16, 'wide': 40, 'is_fixed_direction': 1} [{'x': 0, 'y': 16, 'w': 40}]
The rectangles that haven't been put in yet are： [{'length': 6, 'wide': 7, 'is_fixed_direction': 1}, {'length': 20, 'wide': 5, 'is_fixed_direction': 1}, {'length': 24, 'wide': 24, 'is_fixed_direction': 1}, {'length': 4, 'wide': 7, 'is_fixed_direction': 1}, {'length': 4, 'wide': 4, 'is_fixed_direction': 1}, {'length': 8, 'wide': 7, 'is_fixed_direction': 1}, {'length': 20, 'wide': 4, 'is_fixed_direction': 1}, {'length': 4, 'wide': 5, 'is_fixed_direction': 1}, {'length': 6, 'wide': 7, 'is_fixed_direction': 1}] 

<class 'list'> <class 'list'>
place_first match successfully {'length': 24, 'wide': 24, 'is_fixed_direction': 1, 'area': 576} [{'x': 0, 'y': 40, 'w': 24}, {'x': 24, 'y': 16, 'w': 16}]
The rectangles that haven't been put in yet are： [{'length': 6, 'wide': 7, 'is_fixed_direction': 1, 'area': 42}, {'length': 20, 'wide': 5, 'is_fixed_direct

# N2

In [44]:
def N2():
    total_area = {'length': 50, 'wide': 30}
    N_2 = [
    [23, 9], [19, 4], [12, 21], [6, 4], [7, 13], [9, 4], [4, 6], [23, 6],
    [16, 6], [4, 14], [14, 6], [6, 6], [5, 4], [4, 6], [6, 4], [7, 6],
    [14, 11], [4, 7], [8, 4], [14, 4]
]

    part_list = [{'length': item[1], 'wide': item[0], 'is_fixed_direction': 1} for item in N_2]
    a = BestFitPriority(1)
    
    start_time = time.time()
    
    # Call the controller function to get the packing results
    t = a.controller(part_list, total_area)
    print(t)
    
    print("----------------------------------------------------------\n")
    unsolved, solved = a.one_binNum(part_list, total_area)
    print("Unsolved parts:\n", unsolved)
    print("Solved parts:\n", solved)

    # Calculate the area of solved parts
    solved_area = sum(part['length'] * part['wide'] for part in solved)
    
    # Calculate the utilization
    utilization = solved_area / (total_area['length'] * total_area['wide'])
    print("Utilization:", utilization)
    
    end_time = time.time()  # Record the end time
    execution_time = end_time - start_time  # Calculate the execution time
    print("Execution Time:", execution_time, "seconds")


if __name__ == '__main__':
    N2()

<class 'list'> <class 'list'>
joint_width_fit_first match successfully {'length': 9, 'wide': 23, 'is_fixed_direction': 1} [{'x': 0, 'y': 9, 'w': 23}, {'x': 23, 'y': 0, 'w': 7}]
The rectangles that haven't been put in yet are： [{'length': 4, 'wide': 19, 'is_fixed_direction': 1}, {'length': 21, 'wide': 12, 'is_fixed_direction': 1}, {'length': 4, 'wide': 6, 'is_fixed_direction': 1}, {'length': 13, 'wide': 7, 'is_fixed_direction': 1}, {'length': 4, 'wide': 9, 'is_fixed_direction': 1}, {'length': 6, 'wide': 4, 'is_fixed_direction': 1}, {'length': 6, 'wide': 23, 'is_fixed_direction': 1}, {'length': 6, 'wide': 16, 'is_fixed_direction': 1}, {'length': 14, 'wide': 4, 'is_fixed_direction': 1}, {'length': 6, 'wide': 14, 'is_fixed_direction': 1}, {'length': 6, 'wide': 6, 'is_fixed_direction': 1}, {'length': 4, 'wide': 5, 'is_fixed_direction': 1}, {'length': 6, 'wide': 4, 'is_fixed_direction': 1}, {'length': 4, 'wide': 6, 'is_fixed_direction': 1}, {'length': 6, 'wide': 7, 'is_fixed_direction': 1}, 

# N4

In [45]:
def N4():
    total_area = {'length': 80, 'wide': 80}
    N_4 = [
    [61, 38], [7, 4], [9, 5], [5, 4], [5, 7], [7, 7], [9, 15], [4, 4],
    [4, 4], [32, 31], [4, 4], [5, 4], [8, 10], [4, 4], [5, 5], [32, 4],
    [7, 7], [5, 4], [5, 8], [7, 24], [5, 4], [10, 7], [4, 7], [20, 7],
    [5, 7], [5, 12], [5, 4], [11, 7], [8, 21], [9, 4], [5, 72], [5, 52],
    [5, 4], [9, 7], [5, 12], [9, 33], [4, 8], [9, 34], [4, 4], [29, 4]
]

    part_list = [{'length': item[1], 'wide': item[0], 'is_fixed_direction': 1} for item in N_4]
    a = BestFitPriority(1)
    
    start_time = time.time()
    
    # Call the controller function to get the packing results
    t = a.controller(part_list, total_area)
    print(t)
    
    print("----------------------------------------------------------\n")
    unsolved, solved = a.one_binNum(part_list, total_area)
    print("Unsolved parts:\n", unsolved)
    print("Solved parts:\n", solved)

    # Calculate the area of solved parts
    solved_area = sum(part['length'] * part['wide'] for part in solved)
    
    # Calculate the utilization
    utilization = solved_area / (total_area['length'] * total_area['wide'])
    print("Utilization:", utilization)
    
    end_time = time.time()  # Record the end time
    execution_time = end_time - start_time  # Calculate the execution time
    print("Execution Time:", execution_time, "seconds")


if __name__ == '__main__':
    N4()

<class 'list'> <class 'list'>
place_first match successfully {'length': 38, 'wide': 61, 'is_fixed_direction': 1, 'area': 2318} [{'x': 0, 'y': 38, 'w': 61}, {'x': 61, 'y': 0, 'w': 19}]
The rectangles that haven't been put in yet are： [{'length': 4, 'wide': 7, 'is_fixed_direction': 1, 'area': 28}, {'length': 5, 'wide': 9, 'is_fixed_direction': 1, 'area': 45}, {'length': 4, 'wide': 5, 'is_fixed_direction': 1, 'area': 20}, {'length': 7, 'wide': 5, 'is_fixed_direction': 1, 'area': 35}, {'length': 7, 'wide': 7, 'is_fixed_direction': 1, 'area': 49}, {'length': 15, 'wide': 9, 'is_fixed_direction': 1, 'area': 135}, {'length': 4, 'wide': 4, 'is_fixed_direction': 1, 'area': 16}, {'length': 4, 'wide': 4, 'is_fixed_direction': 1, 'area': 16}, {'length': 31, 'wide': 32, 'is_fixed_direction': 1, 'area': 992}, {'length': 4, 'wide': 4, 'is_fixed_direction': 1, 'area': 16}, {'length': 4, 'wide': 5, 'is_fixed_direction': 1, 'area': 20}, {'length': 10, 'wide': 8, 'is_fixed_direction': 1, 'area': 80}, {'le

# N5

In [46]:
def N5():
    total_area = {'length': 100, 'wide': 100}
    N_5 = [
    [25, 10], [74, 8], [27, 19], [64, 34], [74, 6], [39, 6], [48, 7], [11, 10],
    [8, 10], [14, 6], [6, 10], [26, 6], [27, 6], [6, 10], [8, 10], [31, 10],
    [23, 6], [7, 10], [6, 10], [11, 10], [53, 7], [20, 6], [46, 10], [18, 6],
    [10, 6], [10, 10], [9, 6], [7, 6], [11, 6], [7, 6], [6, 8], [9, 7],
    [47, 7], [10, 7], [7, 6], [25, 10], [26, 8], [36, 6], [6, 6], [6, 7],
    [20, 6], [16, 22], [20, 14], [10, 6], [14, 8], [8, 6], [7, 6], [16, 6],
    [8, 6], [15, 6]
]

    part_list = [{'length': item[1], 'wide': item[0], 'is_fixed_direction': 1} for item in N_5]
    a = BestFitPriority(1)
    
    start_time = time.time()
    
    # Call the controller function to get the packing results
    t = a.controller(part_list, total_area)
    print(t)
    
    print("----------------------------------------------------------\n")
    unsolved, solved = a.one_binNum(part_list, total_area)
    print("Unsolved parts:\n", unsolved)
    print("Solved parts:\n", solved)

    # Calculate the area of solved parts
    solved_area = sum(part['length'] * part['wide'] for part in solved)
    
    # Calculate the utilization
    utilization = solved_area / (total_area['length'] * total_area['wide'])
    print("Utilization:", utilization)
    
    end_time = time.time()  # Record the end time
    execution_time = end_time - start_time  # Calculate the execution time
    print("Execution Time:", execution_time, "seconds")


if __name__ == '__main__':
    N5()

<class 'list'> <class 'list'>
joint_width_fit_first match successfully {'length': 8, 'wide': 74, 'is_fixed_direction': 1} [{'x': 0, 'y': 8, 'w': 74}, {'x': 74, 'y': 0, 'w': 26}]
The rectangles that haven't been put in yet are： [{'length': 10, 'wide': 25, 'is_fixed_direction': 1}, {'length': 19, 'wide': 27, 'is_fixed_direction': 1}, {'length': 34, 'wide': 64, 'is_fixed_direction': 1}, {'length': 6, 'wide': 74, 'is_fixed_direction': 1}, {'length': 6, 'wide': 39, 'is_fixed_direction': 1}, {'length': 7, 'wide': 48, 'is_fixed_direction': 1}, {'length': 10, 'wide': 11, 'is_fixed_direction': 1}, {'length': 10, 'wide': 8, 'is_fixed_direction': 1}, {'length': 6, 'wide': 14, 'is_fixed_direction': 1}, {'length': 10, 'wide': 6, 'is_fixed_direction': 1}, {'length': 6, 'wide': 26, 'is_fixed_direction': 1}, {'length': 6, 'wide': 27, 'is_fixed_direction': 1}, {'length': 10, 'wide': 6, 'is_fixed_direction': 1}, {'length': 10, 'wide': 8, 'is_fixed_direction': 1}, {'length': 10, 'wide': 31, 'is_fixed_dir